# Linear Algebra

## Arrays

> **Note:**
> This section is under development.


## Vector Algebra

As a basic example, consider two orthonormal frames in Euclidean space. The first one is the standard frame defined with the unit matrix $\mathbf{I}$, the second is obtained by rotating $\mathbf{I}$ around the $z$ axis with $90$ degrees. A vector is defined in $\mathbf{I}$ (the source) and we want to know it's components in the rotated frame (the target).

In [11]:
import numpy as np

# the coordinates of a vector in the source frame
arr_source = np.array([3 ** 0.5 / 2, 0.5, 0])
arr_source

array([0.8660254, 0.5      , 0.       ])

### Solution with `NumPy`

We can build up the DCM matrix manually and carry out the operations using NumPy.

In [12]:
from numpy import vstack

# old base vectors in old frame
e1 = np.array([1., 0., 0.])
e2 = np.array([0., 1., 0.])
e3 = np.array([0., 0., 1.])

# new base vectors in old frame
E1 = np.array([0., 1., 0.])
E2 = np.array([-1., 0., 0.])
E3 = np.array([0, 0., 1.])

# direction cosine matrix from old to new
DCM = vstack([E1, E2, E3])
DCM

array([[ 0.,  1.,  0.],
       [-1.,  0.,  0.],
       [ 0.,  0.,  1.]])

From the definition of the DCM matrix, cooridnates of our vector in the target frame can be calculated as:

In [13]:
arr_target = DCM @ arr_source
arr_target

array([ 0.5      , -0.8660254,  0.       ])

### Solution with `SymPy`

SymPy provides a mechanism for obtaining the DCM matrix between two frames:

In [14]:
from sympy.physics.vector import ReferenceFrame

source = ReferenceFrame('source')
target = source.orientnew('target', 'Body', [0, 0, 90*np.pi/180],  'XYZ')
DCM = np.array(target.dcm(source).evalf()).astype(float)
DCM[np.abs(DCM) < 1e-12] = 0.
DCM

array([[ 0.,  1.,  0.],
       [-1.,  0.,  0.],
       [ 0.,  0.,  1.]])

### Solution with `sigmaepsilon.math`

In sigmaepsilon.math, the ReferenceFrame provides the machinery to establish a connection between two frames. First, we create the frames as:

In [15]:
from sigmaepsilon.math.linalg import ReferenceFrame

source = ReferenceFrame(dim=3)  # this is a 3 by 3 identity matrix
target = source.orient_new('Body', [0, 0, 90*np.pi/180],  'XYZ')

The DCM matrix for transformation from the source to the target can be calculated like this:

In [16]:
DCM = source.dcm(target=target)
DCM[np.abs(DCM) < 1e-12] = 0.
DCM

Array([[ 0.,  1.,  0.],
       [-1.,  0.,  0.],
       [ 0.,  0.,  1.]])

or this

In [17]:
DCM = target.dcm(source=source)
DCM[np.abs(DCM) < 1e-12] = 0.
DCM

Array([[ 0.,  1.,  0.],
       [-1.,  0.,  0.],
       [ 0.,  0.,  1.]])

or even this, since in this case the source frame is also the ambient frame:

In [18]:
DCM = target.dcm()
DCM[np.abs(DCM) < 1e-12] = 0.
DCM

array([[ 0.,  1.,  0.],
       [-1.,  0.,  0.],
       [ 0.,  0.,  1.]])

To transform the vector into the target frame, we can use the Vector object directly, and it handles everything in the background.

In [19]:
from sigmaepsilon.math.linalg import Vector

Vector(arr_source, frame=source).show(target)

array([ 0.5      , -0.8660254,  0.       ])

#### Remarks

The following calls all return the identity matrix:

In [20]:
target.dcm(target=target)
target.dcm(source=target)
source.dcm(target=source)
source.dcm(source=source)

Array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

## Tensor Algebra

The library also provides tools to deal with tensors. The implementation has only three limitations:

* The reference frames must be linear.
* All the indices of a tensor must be either covariant or contravariant.
* The sum total of indices used in any kind of operation must be less than or equal to 26. This is because the implementation relies on the `einsum` function in NumPy, and the english alphapet has 26 characters.

Create a Tensor of order 6 in a frame with random components:

In [ ]:
from sigmaepsilon.math.linalg import Tensor, ReferenceFrame
from numpy.random import rand

frame = ReferenceFrame(dim=3)
array = rand(3, 3, 3, 3, 3, 3)
A = Tensor(array, frame=frame)

Get the tensor in the dual frame:

In [ ]:
A_dual = A.dual()

Create an other tensor, in this case a 5th-order one, and calculate their generalized dot product, which is a 9th-order tensor:

In [ ]:
from sigmaepsilon.math.linalg import dot

array = rand(3, 3, 3, 3, 3)
B = Tensor(array, frame=frame)
C = dot(A, B, axes=[0, 0])
assert C.rank == (A.rank + B.rank - 2)